In [49]:
import warnings
warnings.filterwarnings("ignore")
import gmsh
import meshio
import numpy as np
gmsh.initialize()

In [50]:
gmsh.model.add("3D")
L, B, H = 10, 10, 10
r = 1
channel = gmsh.model.occ.addBox(0, 0, 0, L, B, H)
resolution = 0.1

In [51]:
sphere = gmsh.model.occ.addSphere(5, 5, 5, r)

In [52]:
fluid = gmsh.model.occ.cut([(3, channel)], [(3, sphere)])

In [53]:
gmsh.model.occ.synchronize()
volumes = gmsh.model.getEntities(dim=3)

In [54]:
marker = 11
gmsh.model.addPhysicalGroup(volumes[0][0], [volumes[0][1]], marker)
gmsh.model.setPhysicalName(volumes[0][0], marker, "conductor")

In [55]:
surfaces = gmsh.model.occ.getEntities(dim=2)
left_marker = 1
right_marker = 3
sphere_marker = 5
spheres = []
walls = []
for surface in surfaces:
    com = gmsh.model.occ.getCenterOfMass(surface[0], surface[1])
    if np.allclose(com, [0, B/2, H/2]):
        gmsh.model.addPhysicalGroup(surface[0], [surface[1]], left_marker)
        left = surface[1]
        gmsh.model.setPhysicalName(surface[0], left_marker, "left")
    elif np.allclose(com, [L, B/2, H/2]):
        gmsh.model.addPhysicalGroup(surface[0], [surface[1]], right_marker)
        gmsh.model.setPhysicalName(surface[0], right_marker, "right")
        right = surface[1]
    elif np.isclose(com[2], 0) or np.isclose(com[1], B) or np.isclose(com[2], H) or np.isclose(com[1],0):
        walls.append(surface[1])
    else:
        spheres.append(surface[1])
gmsh.model.addPhysicalGroup(2, spheres, sphere_marker)
gmsh.model.setPhysicalName(2, sphere_marker, "sphere")

gmsh.model.mesh.field.add("Distance", 1)
gmsh.model.mesh.field.setNumbers(1, "FacesList", spheres)

gmsh.model.mesh.field.add("Threshold", 2)
gmsh.model.mesh.field.setNumber(2, "IField", 1)
gmsh.model.mesh.field.setNumber(2, "LcMin", resolution)
gmsh.model.mesh.field.setNumber(2, "LcMax", 20*resolution)
gmsh.model.mesh.field.setNumber(2, "DistMin", 0.5*r)
gmsh.model.mesh.field.setNumber(2, "DistMax", r)

gmsh.model.mesh.field.add("Distance", 3)
gmsh.model.mesh.field.setNumbers(3, "FacesList", [left])
gmsh.model.mesh.field.add("Threshold", 4)
gmsh.model.mesh.field.setNumber(4, "IField", 3)
gmsh.model.mesh.field.setNumber(4, "LcMin", 5*resolution)
gmsh.model.mesh.field.setNumber(4, "LcMax", 10*resolution)
gmsh.model.mesh.field.setNumber(4, "DistMin", 0.1)
gmsh.model.mesh.field.setNumber(4, "DistMax", 0.5)

gmsh.model.mesh.field.add("Min", 5)
gmsh.model.mesh.field.setNumbers(5, "FieldsList", [2, 4])
gmsh.model.mesh.field.setAsBackgroundMesh(5)

In [56]:
gmsh.model.occ.synchronize()
gmsh.model.mesh.generate(3)

Info    : Meshing 1D...
Info    : [ 10%] Meshing curve 14 (Circle)
Info    : [ 20%] Meshing curve 16 (Line)
Info    : [ 30%] Meshing curve 17 (Line)
Info    : [ 40%] Meshing curve 18 (Line)
Info    : [ 40%] Meshing curve 19 (Line)
Info    : [ 50%] Meshing curve 20 (Line)
Info    : [ 60%] Meshing curve 21 (Line)
Info    : [ 60%] Meshing curve 22 (Line)
Info    : [ 70%] Meshing curve 23 (Line)
Info    : [ 80%] Meshing curve 24 (Line)
Info    : [ 80%] Meshing curve 25 (Line)
Info    : [ 90%] Meshing curve 26 (Line)
Info    : [100%] Meshing curve 27 (Line)
Info    : Done meshing 1D (Wall 0.148027s, CPU 0.150462s)
Info    : Meshing 2D...
Info    : [  0%] Meshing surface 7 (Sphere, Frontal-Delaunay)
Info    : [ 20%] Meshing surface 8 (Plane, Frontal-Delaunay)
Info    : [ 30%] Meshing surface 9 (Plane, Frontal-Delaunay)
Info    : [ 50%] Meshing surface 10 (Plane, Frontal-Delaunay)
Info    : [ 60%] Meshing surface 11 (Plane, Frontal-Delaunay)
Info    : [ 80%] Meshing surface 12 (Plane, Frontal

In [57]:
gmsh.write("mesh3D.msh")

Info    : Writing 'mesh3D.msh'...
Info    : Done writing 'mesh3D.msh'


In [58]:
def create_mesh(mesh, cell_type, prune_z=False):
    cells = mesh.get_cells_type(cell_type)
    cell_data = mesh.get_cell_data("gmsh:physical", cell_type)
    points = mesh.points[:,:2] if prune_z else mesh.points
    out_mesh = meshio.Mesh(points=points, cells={cell_type: cells}, cell_data={"name_to_read":[cell_data]})
    return out_mesh

In [59]:
mesh_3d = meshio.read("mesh3D.msh")
tetrahedral_mesh = create_mesh(mesh_3d, "tetra")
meshio.write("mesh/10_0_10/mesh_tetr.xdmf", tetrahedral_mesh)

In [64]:
cmd_3d = 'mpirun -n 2 python3 ion_transport.py --working_dir=/home/leshinka/dev/ssb/ --grid_info=10_0_10 --file_shape=10_10'
!{cmd_3d}

done loading tetrahedral mesh
done loading tetrahedral mesh
setting problem..
setting problem..
solving problem..
solving problem..
